Import Libraries and variables

In [123]:
import pandas as pd
import re
import importlib
import vehicle_map_1,vehicle_map_2

importlib.reload(vehicle_map_1)
importlib.reload(vehicle_map_2)
from vehicle_map_1 import vehicle_map_1
from vehicle_map_2 import vehicle_map_2

df=pd.read_csv("./datasets/cleaned_vehicles.csv")

Add,remove and short

In [ ]:
noise_words=["brand new","b/n","brand n","brandnew","new","used","1st","1st owner","petrol","diesel","hybrid","electric","Anniversary",
"second","2nd owner","company maintained","first","owner","owners","manual",
"full option","full options","full-option","full-options","full optioned","full-option","highest spec","high spec","full spec","Pre Order","Moon Roof",
"moonroof","(brandnew)","(brand new)","(moonroof)"]

# columns_to_add={
#     "Body Type":"Model",
#     "Seating Capacity":"Fuel Type",
#     "Ground Clearance":"Seating Capacity",
#     "Ground Clearance":"Ground Clearance (range)",

# }

# for col_name,ref_col in columns_to_add.items():
#     if col_name not in df.columns:
#         col_index=df.columns.get_loc(ref_col)+1
#         df.insert(col_index,col_name,None)
#     else:
#         print(f"{col_name} column already exists")

#----------Modifing Dataset----------
#Lowercase Model
df["Model"]=df["Model"].str.lower().str.strip()
#Sort dataset
df=df.sort_values(by=["Manufacturer","Model","Year"],ascending=[True,True,True])
#----------Rearranging columns----------
#Year column after Model column
cols=list(df.columns)
cols.remove("Year")
cols.insert(cols.index("Model")+1,"Year")
df = df[cols]

# cols=list(df.columns)
# cols.remove("Ground Clearance")
# df=df[cols]

#Fuel Type column after Year column
cols=list(df.columns)
cols.remove("Fuel Type")
cols.insert(cols.index("Year")+1,"Fuel Type")
df = df[cols]


print(f"Saved {len(df)} records to before cleaned_vehicles.csv")
#----------removing noise words from Model column----------
for word in noise_words:
    df["Model"]=df["Model"].str.replace(word,"",regex=True)
df["Model"]=df["Model"].str.replace(r"\s+"," ",regex=True).str.strip()

#----------removing duplicates----------
df=df.drop_duplicates(subset=["Manufacturer","Model","Year"],keep="first")

#----------Replace words in Model column----------
mask = (df["Manufacturer"].str.strip().str.lower() == "ford") & \
       (df["Model"].str.lower().str.contains("raptor ranger"))
df.loc[mask, "Model"] = df.loc[mask, "Model"].str.replace(
    r"raptor\s*ranger", "ranger raptor", regex=True, case=False
)
#----------Remove floating points----------
df["Seating Capacity"] = df["Seating Capacity"].astype("Int64")

Saved 2683 records to before cleaned_vehicles.csv


In [125]:
vehicle_maps={**vehicle_map_1,**vehicle_map_2}

for brand, attributes in vehicle_maps.items():
    for model, seating_capacity in attributes["seating_capacity"].items():
        df.loc[
            (df["Manufacturer"].str.strip().str.lower() == brand.strip().lower())
            & (df["Model"].str.contains(model, case=False, regex=False)),
            "Seating Capacity",
        ] = seating_capacity

    for model, btype in attributes["body_type"].items():
        mask = (df["Manufacturer"] == brand) & (df["Model"].str.lower().eq(model))
        df.loc[mask, "Body Type"] = btype
        mask = (df["Manufacturer"] == brand) & (
            df["Model"].str.lower().str.contains(model)
        )
        df.loc[mask & df["Body Type"].isna(), "Body Type"] = btype
    
    
 


    print(f"Total {brand} cars: {df[df['Manufacturer'] == brand].shape[0]}")

Total Audi cars: 83
Total BMW cars: 132
Total BYD cars: 6
Total Chery cars: 6
Total Chevrolet cars: 1
Total DFSK cars: 18
Total Daihatsu cars: 105
Total Datsun cars: 2
Total Dongfeng cars: 1
Total Fiat cars: 1
Total Ford cars: 55
Total Honda cars: 211
Total Hyundai cars: 40
Total Isuzu cars: 1
Total JAC cars: 2
Total Jaecoo cars: 3
Total JMEV cars: 1
Total Jaguar cars: 5
Total Jeep cars: 9
Total Kia cars: 70
Total Land Rover cars: 112
Total Lexus cars: 16
Total MG cars: 8
Total Mahindra cars: 13
Total Maruti Suzuki cars: 51
Total Mazda cars: 23
Total Mercedes Benz cars: 136
Total Micro cars: 39
Total Mitsubishi cars: 108
Total Nissan cars: 140
Total Perodua cars: 15
Total Peugeot cars: 22
Total Porsche cars: 6
Total Renault cars: 6
Total SsangYong  cars: 0
Total Subaru cars: 3
Total Suzuki cars: 164
Total Tata cars: 22
Total Tesla cars: 8
Total Toyota cars: 1010


In [126]:
df.to_csv("./datasets/cleaned_vehicles.csv",index=False)
print(f"Saved {len(df)} records to cleaned_vehicles.csv")

Saved 2681 records to cleaned_vehicles.csv
